In [204]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\nicol\AppData\Local\Temp\ipykernel_10288\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [205]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [206]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [207]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [208]:
# Your code
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

X_train.head()

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
Name: text, dtype: object

## Data Preprocessing

In [209]:
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [210]:
# Your code

import re

def preprocess_text(text):
    # Remove HTML tags
    text = re.sub('<[^<]+?>', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'\b\w', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b'
    text = text.replace('b', '')
    
    # Convert to Lowercase
    text = text.lower()
    
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [211]:
# Your code

# Apply the preprocessing function to the text column

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [212]:
X_train.head()

29      egards r elson mithkindly eply e n y rivate m...
535     ave ot een le o each scar his m e re upposed ...
695     uma edin im hecking ith at n he ill ork ith a...
557               an ave t nnounced ere n onday ant oday
836     ank f fricaagence an edro p an edro ote ivoir...
Name: text, dtype: object

## Now let's work on removing stopwords
Remove the stopwords.

In [213]:
# Your code

from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

X_train = X_train.apply(remove_stopwords)
X_test = X_test.apply(remove_stopwords)

print(X_train.head())

29     egards r elson mithkindly eply e n rivate mail...
535              ave ot een le scar e upposed end eceive
695    uma edin im hecking ith n ill ork ith ack ake ...
557                    ave nnounced ere n onday ant oday
836    ank f fricaagence edro p edro ote ivoirewest f...
Name: text, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [214]:
# Your code

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return' '.join([lemmatizer.lemmatize(word) for word in text.split()])

X_train = X_train.apply(lemmatize_text)

X_test = X_test.apply(lemmatize_text)

X_train.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


29     egards r elson mithkindly eply e n rivate mail...
535              ave ot een le scar e upposed end eceive
695    uma edin im hecking ith n ill ork ith ack ake ...
557                    ave nnounced ere n onday ant oday
836    ank f fricaagence edro p edro ote ivoirewest f...
Name: text, dtype: object

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [215]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re

# Configure the CountVectorizer
vectorizer = CountVectorizer(
    stop_words='english',  # Remove English stop words
    lowercase=True,        # Convert text to lowercase
    token_pattern=r'\b[a-zA-Z]+\b',  # Only consider words with letters
    max_features=1000      # Limit the number of features
)

# Transform the text into a Bag of Words (BoW) matrix
X_train_bow = vectorizer.fit_transform(X_train)
feature_names = vectorizer.get_feature_names_out()

# Separate ham and spam messages
ham_indices = y_train == 0  # Assuming ham is labeled as 0
spam_indices = y_train == 1  # Assuming spam is labeled as 1

X_train_ham_bow = X_train_bow[ham_indices]  # BoW for ham messages
X_train_spam_bow = X_train_bow[spam_indices]  # BoW for spam messages

# Function to get the top N words
def get_top_words(bow_features, feature_names, n=10):
    # Sum the word counts along axis 0 (columns)
    word_counts = np.asarray(bow_features.sum(axis=0)).flatten()
    # Create a list of (word, count) tuples
    word_counts = list(zip(feature_names, word_counts))
    # Sort the list by count in descending order and select the top N words
    sorted_words = sorted(word_counts, key=lambda x: x[1], reverse=True)[:n]
    return sorted_words

# Get and print the top 10 words for ham and spam
top_ham_words = get_top_words(X_train_ham_bow, feature_names, n=10)
top_spam_words = get_top_words(X_train_spam_bow, feature_names, n=10)

print("Top 10 words in HAM messages:")
for word, count in top_ham_words:
    print(f"{word}: {count}")

print("\nTop 10 words in SPAM messages:")
for word, count in top_spam_words:
    print(f"{word}: {count}")



Top 10 words in HAM messages:
n: 917
f: 775
nd: 761
e: 526
hat: 420
ou: 248
ith: 206
ut: 198
ill: 195
r: 194

Top 10 words in SPAM messages:
f: 4310
n: 3657
e: 3346
nd: 3342
ou: 2467
ill: 1512
hat: 1455
ith: 1043
ave: 805
oney: 795


In [216]:
X_train.head()

29     egards r elson mithkindly eply e n rivate mail...
535              ave ot een le scar e upposed end eceive
695    uma edin im hecking ith n ill ork ith ack ake ...
557                    ave nnounced ere n onday ant oday
836    ank f fricaagence edro p edro ote ivoirewest f...
Name: text, dtype: object

## Extra features

In [217]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).

# Convert X_train and X_test to DataFrames
X_train = pd.DataFrame(X_train, columns=['text'])
X_test = pd.DataFrame(X_test, columns=['text'])

# List of currency symbols and suspicious words
money_symbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Add indicators to the training DataFrame
X_train['money_mark'] = X_train['text'].str.contains(money_symbol_list, case=False, regex=True) * 1
X_train['suspicious_words'] = X_train['text'].str.contains(suspicious_words, case=False, regex=True) * 1
X_train['text_len'] = X_train['text'].apply(lambda x: len(x))

# Add indicators to the test DataFrame
X_test['money_mark'] = X_test['text'].str.contains(money_symbol_list, case=False, regex=True) * 1
X_test['suspicious_words'] = X_test['text'].str.contains(suspicious_words, case=False, regex=True) * 1
X_test['text_len'] = X_test['text'].apply(lambda x: len(x))

# Display the first few rows of the training DataFrame
X_train.head()

,text,money_mark,suspicious_words,text_len
29,egards r elson mithkindly eply e n rivate mail...,1,0,72
535,ave ot een le scar e upposed end eceive,1,0,39
695,uma edin im hecking ith n ill ork ith ack ake ...,1,0,82
557,ave nnounced ere n onday ant oday,1,0,33
836,ank f fricaagence edro p edro ote ivoirewest f...,1,0,1094


## How would you create a Bag of Words with the CountVectorizer method?

In [222]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer

# Configure the CountVectorizer

vectorizer = CountVectorizer(
    stop_words='english',  # Remove English stop words
    lowercase=True,        # Convert text to lowercase
    token_pattern=r'\b[a-zA-Z]+\b',  # Only consider words with letters
    max_features=1000      # Limit the number of features
)

# Transform the text into a Bag of Words (BoW) matrix

X_train_bow = vectorizer.fit_transform(X_train['text'])
feature_names = vectorizer.get_feature_names_out()

print("BoW matrix shape:", X_train_bow.shape)

print("Top 10 most frequent words:")

top_words = get_top_words(X_train_bow, feature_names, n=10)

for word, count in top_words:
    print(f"{word}: {count}")
    print()




BoW matrix shape: (800, 1000)
Top 10 most frequent words:
f: 5085

n: 4574

nd: 4103

e: 3872

ou: 2715

hat: 1875

ill: 1707

ith: 1249

ave: 978

r: 868



## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [223]:
# Your code

from sklearn.feature_extraction.text import TfidfVectorizer

# Configure the TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words='english',  # Remove English stop words
    lowercase=True,        # Convert text to lowercase
    token_pattern=r'\b[a-zA-Z]+\b',  # Only consider words with letters
    max_features=1000      # Limit the number of features
)

# Vectorize the text into a TF-IDF matrix

X_train_tfidf = vectorizer.fit_transform(X_train['text'])
feature_names = vectorizer.get_feature_names_out()

print("TF-IDF matrix shape:", X_train_tfidf.shape)

print("Top 10 most frequent words:")

top_words = get_top_words(X_train_tfidf, feature_names, n=10)

for word, count in top_words:
    print(f"{word}: {count}")
    print()



TF-IDF matrix shape: (800, 1000)
Top 10 most frequent words:
f: 119.50810507439287

n: 113.92349649248122

e: 107.81892920087421

nd: 101.39581837876162

ou: 79.50878441793752

ill: 56.68349110129105

hat: 49.438459737047225

ith: 37.310789811430766

yi: 33.127904855234725

ave: 31.758954257137198



### Extra Task (optional) - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

Use a MultinimialNB with default parameters.

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [230]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(data['text'])

X_train, X_test, y_train, y_test = train_test_split(X, data['label'], test_size=0.2, random_state=42)

model = MultinomialNB()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

cv_scores = cross_val_score(model, X, data['label'], cv=5)
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98       125
           1       0.93      1.00      0.96        75

    accuracy                           0.97       200
   macro avg       0.96      0.98      0.97       200
weighted avg       0.97      0.97      0.97       200

Confusion Matrix:
[[119   6]
 [  0  75]]
Cross-Validation Accuracy Scores: [0.965 0.96  0.97  0.97  0.945]
Mean CV Accuracy: 0.962
